# Data exploration

I have exported my data from Polar watch. Exploring what is there 

In [11]:
import glob
import os
import json
import random
import pandas as pd

Define consts

In [2]:
DATA_DIR = "/Users/arbatov/gitRepo/gpx-data/data"

## Get the data

Load the files

In [3]:
json_files = glob.glob(os.path.join(DATA_DIR,"*.json"))

How many files?

In [4]:
print(len(json_files))

815


How big are all of the files?

In [13]:
nbytes = sum(d.stat().st_size for d in os.scandir(DATA_DIR) if d.is_file())
print(float(nbytes) / 1024**3, "GB")

1.7585922731086612 GB


## Overview of data

I need to sample data

In [19]:
def get_sample(sample_size = 200):  
    file_contents = []
    
    sampled_json_files = random.sample(json_files, sample_size)
    for json_file in sampled_json_files:
        with open(json_file) as file:
            j = json.load(file)
            
            file_contents.append([
                os.path.basename(json_file),
                j['exercises'][0]['sport'],
                j['startTime'],
                j['stopTime'],
                if 'distance' in j: j['distance'],
                j['duration'],
                j['maximumHeartRate'],
                j['averageHeartRate']
            ])

    return file_contents


data = get_sample(sample_size=200)
data_df = pd.DataFrame(data, columns=[
    'File_Name', 
    'Sport',
    'Start_Time',
    'Stop_Time',
    'Distance',
    'Duration',
    'Maximum_Heart_Rate',
    'Average_Heart_Rate'
])
data_df.head()

KeyError: 'distance'

Get the data types

In [17]:
data_df.dtypes

File_Name    object
Sport        object
dtype: object

What kind of activities are there?

In [8]:
activities = {}

file_contents = get_sample(sample_size=200)
for content in file_contents:
    sport = content['exercises'][0]['sport']
    if sport in activities:
        activities[sport] += 1
    else:
        activities[sport] = 1

sorted_activities = dict(sorted(
    activities.items(), 
    key=lambda activity: activity[1],
    reverse=True
))            
for sport, count in sorted_activities.items():
    print(sport, count)

RUNNING 102
STRETCHING 28
OPEN_WATER_SWIMMING 25
STRENGTH_TRAINING 15
CYCLING 13
POOL_SWIMMING 10
INDOOR_CYCLING 3
TREADMILL_RUNNING 2
WALKING 1
YOGA 1


Get activities of specific sport

In [9]:
running_acitivities = get_sample("RUNNING", 200)
print(len(running_acitivities))

97
